In [1]:
"""
Purpose: New skeletonization method that does
not use a recursive method but simply uses the followoing
algorithm: 
- Do mesh splitting to find which mesh pieces aren't connect to
  the soma
0) Do Soma Extraction and split the meshes from there
For Each significant mesh that was split off in beginning
1) Poisson Surface Reconstruction
2) CGAL skeletonization of all signfiicant pieces
3) Using CGAL skeleton, find the leftover mesh not skeletonized
4) Do surface reconstruction on the parts that are left over
- with some downsampling
5) Stitch the skeleton 


---- Afterwards stitching:
1) Compute the soma mesh center point
2) For meshes that were originally connected to soma
a. Find the closest skeletal point to soma center
b. Add an edge from closest point to soma center
3) Then do stitching algorithm on all of remaining disconnected
    skeletons

"""

"\nPurpose: New skeletonization method that does\nnot use a recursive method but simply uses the followoing\nalgorithm: \n- Do mesh splitting to find which mesh pieces aren't connect to\n  the soma\n0) Do Soma Extraction and split the meshes from there\nFor Each significant mesh that was split off in beginning\n1) Poisson Surface Reconstruction\n2) CGAL skeletonization of all signfiicant pieces\n3) Using CGAL skeleton, find the leftover mesh not skeletonized\n4) Do surface reconstruction on the parts that are left over\n- with some downsampling\n5) Stitch the skeleton \n\n\n---- Afterwards stitching:\n1) Compute the soma mesh center point\n2) For meshes that were originally connected to soma\na. Find the closest skeletal point to soma center\nb. Add an edge from closest point to soma center\n3) Then do stitching algorithm on all of remaining disconnected\n    skeletons\n\n"

In [2]:
from pykdtree.kdtree import KDTree
import time
import trimesh
import numpy as np
from pathlib import Path

import time
import os
import pathlib

from tqdm.notebook import tqdm

In [3]:
import meshlab
from importlib import reload
meshlab = reload(meshlab)
from meshlab import Decimator , Poisson
import skeleton_utils as sk

In [4]:
#setting the calcification edge size


# Importing Example Mesh and Example Soma

In [5]:
import soma_extraction_utils as soma_utils
from pathlib import Path
import trimesh

In [6]:
def load_somas(segment_id,main_mesh_total):
    try:
        current_soma = trimesh.load_mesh("./Dustin_soma.off")
        return [current_soma]
    except:
        print("No Soma currently available so must compute own")
        (total_soma_list, 
             run_time, 
             total_soma_list_sdf) = soma_utils.extract_soma_center(
                                segment_id,
                                main_mesh_total.vertices,
                                main_mesh_total.faces,
                                outer_decimation_ratio= 0.25,
                                large_mesh_threshold = 60000,
                                large_mesh_threshold_inner = 40000,
                                soma_width_threshold = 0.32,
                                soma_size_threshold = 20000,
                               inner_decimation_ratio = 0.25,
                               volume_mulitplier=7,
                               side_length_ratio_threshold=3,
                                soma_size_threshold_max=192000,
                                delete_files=True
            )
        return total_soma_list
    else:
        return []

In [7]:
segment_id = 12345

# Load soma mesh
# #loads in the Dustin mesh
main_mesh_path = Path("./Dustin.off")
main_mesh_total = trimesh.load_mesh(str(main_mesh_path.absolute()))

soma_mesh_list = load_somas(segment_id,main_mesh_total)
print(f"Soma List = {soma_mesh_list}")

soma_mesh_list_centers = [np.array(np.mean(k.vertices,axis=0)).astype("float")
                           for k in soma_mesh_list]
print(f"soma_mesh_list_centers = {soma_mesh_list_centers}")

if len(soma_mesh_list) <= 0:
    print(f"**** No Somas Found for Mesh {segment_id}")
    


Soma List = [<trimesh.Trimesh(vertices.shape=(1864, 3), faces.shape=(3640, 3))>]
soma_mesh_list_centers = [array([1326076.34924893,  732678.79350858,  884152.51410944])]


In [8]:
from importlib import reload
sk = reload(sk)

# Do mesh splitting to find which mesh pieces aren't connect to the soma

In [9]:
split_meshes = sk.split_significant_pieces(
                            main_mesh_total,
                            significance_threshold=15,
                            print_flag=False)
len(split_meshes)

110

# Find which bounding box contains the soma

In [10]:
"""
Pseudocode: 
For all meshes in list
1) compute soma center
2) Find all the bounding boxes that contain the soma center
3) Find the mesh with the closest distance from 
   one vertex to the soma center and tht is winner
"""
containing_mesh_indices=dict([(i,[]) for i,sm_c in enumerate(soma_mesh_list_centers)])
for k,sm_center in enumerate(soma_mesh_list_centers):

    viable_meshes = [j for j,m in enumerate(split_meshes) 
             if trimesh.bounds.contains(m.bounds,sm_center.reshape(-1,3))
                    ]
    if len(viable_meshes) == 0:
        raise Exception(f"The Soma {k} with {sm_center} was not contained in any of the boundying boxes")
    elif len(viable_meshes) == 1:
        containing_mesh_indices[k] = viable_meshes[0]
    else:
        #find which mesh is closer to the soma midpoint
        min_distances_to_soma = []
        for v_i in viable_meshes:
            # build the KD Tree
            viable_neuron_kdtree = KDTree(soma_mesh_list[v_i].vertices)
            distances,closest_node = viable_neuron_kdtree.query(sm_centers.reshape(-1,3))
            min_distances_to_soma.append(np.min(distances))
        print(f"min_distances_to_soma = {min_distances_to_soma}")
        containing_mesh_indices[k] = np.argmin(min_distances_to_soma)

In [11]:
non_soma_touching_meshes = [m for i,m in enumerate(split_meshes)
                     if i not in list(containing_mesh_indices.values())]
len(non_soma_touching_meshes)
soma_touching_meshes = dict([(i,split_meshes[m_i]) 
                             for i,m_i in containing_mesh_indices.items()])
soma_touching_meshes

{0: <trimesh.Trimesh(vertices.shape=(325120, 3), faces.shape=(651866, 3))>}

# Part 1: For each soma containing mesh: Do Skeletonization and stitching

# - util functions for skeletonization

In [12]:
def subtract_soma(current_soma,main_mesh):
    
    start_time = time.time()
    face_midpoints_soma = np.mean(current_soma.vertices[current_soma.faces],axis=1)


    curr_mesh_bbox_restriction,faces_bbox_inclusion = (
                    sk.bbox_mesh_restrcition(main_mesh,
                                             current_soma.bounds,
                                            mult_ratio=1.3)
    )

    face_midpoints_neuron = np.mean(curr_mesh_bbox_restriction.vertices[curr_mesh_bbox_restriction.faces],axis=1)

    soma_kdtree = KDTree(face_midpoints_soma)

    distances,closest_node = soma_kdtree.query(face_midpoints_neuron)

    distance_threshold = 550
    distance_passed_faces  = distances<distance_threshold

    faces_to_keep = np.array(list(set(np.arange(0,len(main_mesh.faces))).difference(set(faces_bbox_inclusion[distance_passed_faces]))))
    without_soma_mesh = main_mesh.submesh([faces_to_keep],append=True)

    #get the significant mesh pieces
    mesh_pieces = sk.split_significant_pieces(without_soma_mesh,significance_threshold=200)
    print(f"Total Time for soma mesh cancellation = {np.round(time.time() - start_time,3)}")
    return mesh_pieces

In [13]:
"""
Pseudocode: 
0) Do Soma Extraction and split the meshes from there
For Each significant mesh that was split off in beginning
1) Poisson Surface Reconstruction
2) CGAL skeletonization of all signfiicant pieces 
    (if above certain size ! threshold) 
            --> if not skip straight to surface skeletonization
3) Using CGAL skeleton, find the leftover mesh not skeletonized
4) Do surface reconstruction on the parts that are left over
- with some downsampling
5) Stitch the skeleton 

"""

'\nPseudocode: \n0) Do Soma Extraction and split the meshes from there\nFor Each significant mesh that was split off in beginning\n1) Poisson Surface Reconstruction\n2) CGAL skeletonization of all signfiicant pieces \n    (if above certain size ! threshold) \n            --> if not skip straight to surface skeletonization\n3) Using CGAL skeleton, find the leftover mesh not skeletonized\n4) Do surface reconstruction on the parts that are left over\n- with some downsampling\n5) Stitch the skeleton \n\n'

# - setting up the paths for data writing

In [14]:
from shutil import rmtree
from pathlib import Path

mesh_base_path=Path("./Dustin_vp6/")
current_name="Dustin"

if mesh_base_path.exists():
    rmtree(str(mesh_base_path.absolute()))
mesh_base_path.mkdir(parents=True,exist_ok=True)
print(list(mesh_base_path.iterdir()))

[]


In [15]:
def skeletonize_connected_branch(current_mesh,
                        output_folder="./temp",
                        delete_temp_files=True,
                        name="None",
                        surface_reconstruction_size=50,
                        n_surface_downsampling = 1,
                        n_surface_samples=1000,
                        skeleton_print=False,
                        mesh_subtraction_distance_threshold=3000,
                        mesh_subtraction_buffer=50,
                        max_stitch_distance = 18000,
                        current_min_edge = 200
                        ):
    """
    Purpose: To take a mesh and construct a full skeleton of it
    (Assuming the Soma is already extracted)
    
    1) Poisson Surface Reconstruction
    2) CGAL skeletonization of all signfiicant pieces 
        (if above certain size ! threshold) 
                --> if not skip straight to surface skeletonization
    3) Using CGAL skeleton, find the leftover mesh not skeletonized
    4) Do surface reconstruction on the parts that are left over
    - with some downsampling
    5) Stitch the skeleton 
    """
    
    #check that the mesh is all one piece
    current_mesh_splits = sk.split_significant_pieces(current_mesh,
                               significance_threshold=1)
    if len(current_mesh_splits) > 1:
        raise Exception(f"The mesh passed has {len(current_mesh_splits)} pieces")

    # check the size of the branch and if small enough then just do
    # Surface Skeletonization
    if len(current_mesh.faces) < surface_reconstruction_size:
        #do a surface skeletonization
        print("Doing skeleton surface reconstruction")
        surf_sk = sk.generate_surface_skeleton(current_mesh.vertices,
                                    current_mesh.faces,
                                    surface_samples=n_surface_samples,
                                             n_surface_downsampling=n_surface_downsampling )
        return surf_sk
    else:
    
        #if can't simply do a surface skeletonization then 
        #use cgal method that requires temp folder

        if type(output_folder) != type(Path()):
            output_folder = Path(str(output_folder))
            output_folder.mkdir(parents=True,exist_ok=True)
            
        # CGAL Step 1: Do Poisson Surface Reconstruction
        Poisson_obj = Poisson(output_folder,overwrite=True)
        

        skeleton_start = time.time()
        print("     Starting Screened Poisson")
        new_mesh,output_subprocess_obj = Poisson_obj(   
                                    vertices=current_mesh.vertices,
                                     faces=current_mesh.faces,
                                    mesh_filename=name + ".off",
                                     return_mesh=True,
                                     delete_temp_files=False,
                                    )
        print(f"-----Time for Screened Poisson= {time.time()-skeleton_start}")
            
        #2) Filter away for largest_poisson_piece:
        mesh_pieces = sk.split_significant_pieces(new_mesh,
                                            significance_threshold=surface_reconstruction_size)
        
        if skeleton_print:
            print(f"Signifiant mesh pieces of {surface_reconstruction_size} size "
                 f"after poisson = {len(mesh_pieces)}")
        skeleton_ready_for_stitching = np.array([])
        skeleton_files = [] # to be erased later on if need be
        if len(mesh_pieces) <= 0:
            if skeleton_print:
                print("No signficant skeleton pieces so just doing surface skeletonization")
            # do surface skeletonization on all of the pieces
            surface_mesh_pieces = sk.split_significant_pieces(new_mesh,
                                            significance_threshold=2)
            
            #get the skeletons for all those pieces
            current_mesh_skeleton_list = [
                sk.generate_surface_skeleton(p.vertices,
                                    p.faces,
                                    surface_samples=n_surface_samples,
                                    n_surface_downsampling=n_surface_downsampling )
                for p in surface_mesh_pieces
            ]
            
            skeleton_ready_for_stitching = sk.stack_skeletons(current_mesh_skeleton_list)
            
            #will stitch them together later
        else: #if there are parts that can do the cgal skeletonization
            skeleton_start = time.time()
            print("     Starting Calcification")
            for zz,piece in enumerate(mesh_pieces):
                current_mesh_path = output_folder / f"{name}_{zz}"
                
                written_path = sk.write_neuron_off(piece,current_mesh_path)
                
                #print(f"Path sending to calcification = {written_path[:-4]}")
                returned_value, sk_file_name = sk.calcification(written_path,
                                                               min_edge_length = current_min_edge)
                #print(f"Time for skeletonizatin = {time.time() - skeleton_start}")
                skeleton_files.append(sk_file_name)
                
            if skeleton_print:
                print(f"-----Time for Running Calcification = {time.time()-skeleton_start}")
            
            #collect the skeletons and subtract from the mesh
            
            significant_poisson_skeleton = sk.read_skeleton_edges_coordinates(skeleton_files)
            
            if len(significant_poisson_skeleton) > 0:
                boolean_significance_threshold=5

                print(f"Before mesh subtraction number of skeleton edges = {significant_poisson_skeleton.shape[0]+1}")
                mesh_pieces_leftover =  sk.mesh_subtraction_by_skeleton(current_mesh,
                                                            significant_poisson_skeleton,
                                                            buffer=mesh_subtraction_buffer,
                                                            bbox_ratio=1.2,
                                                            distance_threshold=significant_poisson_skeleton,
                                                            significance_threshold=boolean_significance_threshold,
                                                            print_flag=False
                                                           )

                # *****adding another significance threshold*****
                leftover_meshes_sig = [k for k in mesh_pieces_leftover if len(k.faces) > 50]
                leftover_meshes = sk.combine_meshes(leftover_meshes_sig)
            else:
                print("No recorded skeleton so skiipping"
                     " to surface skeletonization")
                leftover_meshes_sig = [current_mesh]
    
            leftover_meshes_sig_surf_sk = []
            for m in tqdm(leftover_meshes_sig):
                surf_sk = sk.generate_surface_skeleton(m.vertices,
                                               m.faces,
                                               surface_samples=n_surface_samples,
                                    n_surface_downsampling=n_surface_downsampling )
                if len(surf_sk) > 0:
                    leftover_meshes_sig_surf_sk.append(surf_sk)
            leftovers_stacked = sk.stack_skeletons(leftover_meshes_sig_surf_sk)
            #print(f"significant_poisson_skeleton = {significant_poisson_skeleton}")
            #print(f"leftover_meshes_sig_surf_sk = {leftover_meshes_sig_surf_sk}")
            skeleton_ready_for_stitching = sk.stack_skeletons([significant_poisson_skeleton,leftovers_stacked])
            
        #now want to stitch together whether generated from 
        if skeleton_print:
            print(f"After cgal process the un-stitched skeleton has shape {skeleton_ready_for_stitching.shape}")
        
        stitched_skeletons_full = sk.stitch_skeleton(
                                                  skeleton_ready_for_stitching,
                                                  max_stitch_distance=max_stitch_distance,
                                                  stitch_print = False,
                                                  main_mesh = []
                                                )
        stitched_skeletons_full_cleaned = sk.clean_skeleton(stitched_skeletons_full)
        
        # erase the skeleton files if need to be
        if delete_temp_files:
            for sk_fi in skeleton_files:
                if Path(sk_fi).exists():
                    Path(sk_fi).unlink()
        
        # if created temp folder then erase if empty
        if str(output_folder.absolute()) == str(Path("./temp").absolute()):
            print("The process was using a temp folder")
            if len(list(output_folder.iterdir())) == 0:
                print("Temp folder was empty so deleting it")
                if output_folder.exists():
                    rmtree(str(output_folder.absolute()))
        
        return stitched_skeletons_full_cleaned    

In [16]:
def soma_skeleton_stitching(total_soma_skeletons,soma_mesh):
    """
    Purpose: Will stitch together the meshes that are touching
    the soma 
    
    Pseudocode: 
    1) Compute the soma mesh center point
    2) For meshes that were originally connected to soma
    a. Find the closest skeletal point to soma center
    b. Add an edge from closest point to soma center
    3) Then do stitching algorithm on all of remaining disconnected
        skeletons
    
    
    """
    # 1) Compute the soma mesh center point
    soma_center = np.mean(soma_mesh.vertices,axis=0)
    
    soma_connecting_skeleton = []
    for skel in total_soma_skeletons:
        #get the unique vertex points
        unique_skeleton_nodes = np.unique(skel.reshape(-1,3),axis=0)
        
        # a. Find the closest skeletal point to soma center
        # b. Add an edge from closest point to soma center
        mesh_tree = KDTree(unique_skeleton_nodes)
        distances,closest_node = mesh_tree.query(soma_center.reshape(-1,3))
        closest_skeleton_vert = unique_skeleton_nodes[closest_node[np.argmin(distances)]]
        soma_connecting_skeleton.append(np.array([closest_skeleton_vert,soma_center]).reshape(-1,2,3))
    
    print(f"soma_connecting_skeleton[0].shape = {soma_connecting_skeleton[0].shape}")
    print(f"total_soma_skeletons[0].shape = {total_soma_skeletons[0].shape}")
    # stith all of the ekeletons together
    soma_stitched_sk = sk.stack_skeletons(total_soma_skeletons + soma_connecting_skeleton)
    
    return soma_stitched_sk

In [17]:
#cm.calcification("/notebooks3/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum/Platinum_Skeletonization_vp3/Dustin_vp4/Dustin_soma_0_branch_0_0")

In [18]:
sk = reload(sk)

soma_touching_time = time.time()
soma_touching_meshes_skeletons = []
for s_i,main_mesh in soma_touching_meshes.items():
    #Do the mesh subtraction to get the disconnected pieces
    current_soma = soma_mesh_list[s_i]
    
    mesh_pieces = subtract_soma(current_soma,main_mesh)
    print(f"mesh_pieces after the soma subtraction = {len(mesh_pieces)}")
    #get each branch skeleton
    total_soma_skeletons = []
    for dendrite_index,picked_dendrite in enumerate(mesh_pieces):
        dendrite_name=current_name + f"_soma_{s_i}_branch_{dendrite_index}"
        print(f"\n\nWorking on {dendrite_name}")
        stitched_dendrite_skeleton = skeletonize_connected_branch(picked_dendrite,
                                                       output_folder=mesh_base_path,
                                                       name=dendrite_name,
                                                        skeleton_print = True)
        
        if len(stitched_dendrite_skeleton)<=0:
            print(f"*** Dendrite {dendrite_index} did not have skeleton computed***")
        else: 
            total_soma_skeletons.append(stitched_dendrite_skeleton)
            
#print(f"Total time for soma touching skeletons: {time.time() - soma_touching_time}")

Total Time for soma mesh cancellation = 1.216
mesh_pieces after the soma subtraction = 7


Working on Dustin_soma_0_branch_0
     Starting Screened Poisson
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 925
xvfb-run -n 925 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/Dustin_vp6/Dustin_soma_0_branch_0.off -o /notebooks/Platinum_Skeletonization_vp3/Dustin_vp6/Dustin_soma_0_branch_0_poisson.off -s /notebooks/Platinum_Skeletonization_vp3/Dustin_vp6/poisson_872526.mls
-----Time for Screened Poisson= 35.17968153953552


face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

Signifiant mesh pieces of 50 size after poisson = 502
     Starting Calcification
-----Time for Running Calcification = 19.00791907310486
Before mesh subtraction number of skeleton edges = 4441
Inside mesh subtraction, len(main_mesh_bbox_restricted.faces) = 247794



Total Mesh subtraction time = 201.0881



After cgal process the un-stitched skeleton has shape (5650, 2, 3)
len_subgraphs AT BEGINNING of the loop


all graph is one component!

Total time for skeleton stitching = 2.668152093887329


TypeError: clean_skeleton() missing 1 required positional argument: 'distance_func'

In [ ]:
print(f"Total time for soma touching skeletons: {time.time() - soma_touching_time}")

In [ ]:
len(total_soma_skeletons)

# stitch the skeletons together with the soma

In [ ]:
soma_stitched_skeleton = soma_skeleton_stitching(total_soma_skeletons,current_soma)

# Visualize the stitched skeleton for the parts touching the soma

In [ ]:
soma_stitched_skeleton.shape

In [ ]:
sk.graph_skeleton_and_mesh(main_mesh_total.vertices,
                           main_mesh_total.faces,
                           edge_coordinates=soma_stitched_skeleton)

# Do Skeletonization of all non-soma touching branches

In [ ]:
len(non_soma_touching_meshes)

In [ ]:
sk = reload(sk)

In [ ]:
non_soma_time = time.time()

non_soma_touching_meshes

total_non_soma_skeletons = []
for j,picked_non_soma_branch in enumerate(non_soma_touching_meshes):
#     if j<66:
#         continue
    dendrite_name=current_name + f"_non_soma_{j}"
    print(f"\n\nWorking on {dendrite_name}")
    stitched_dendrite_skeleton = skeletonize_connected_branch(picked_non_soma_branch,
                                                   output_folder=mesh_base_path,
                                                   name=dendrite_name,
                                                    skeleton_print = True)

    if len(stitched_dendrite_skeleton)<=0:
        print(f"*** Dendrite {dendrite_index} did not have skeleton computed***")
    else: 
        total_non_soma_skeletons.append(stitched_dendrite_skeleton)


print(f"Time for non-soma skeletons = {time.time() - non_soma_time}")

# Stitch the All Soma touching and non soma touching branches

In [ ]:
stacked_non_soma_skeletons = sk.stack_skeletons(total_non_soma_skeletons)
stacked_soma_skeletons = sk.stack_skeletons([soma_stitched_skeleton])
whole_skeletons_for_stitching = sk.stack_skeletons([stacked_non_soma_skeletons,stacked_soma_skeletons])

final_skeleton_pre_clean = sk.stitch_skeleton(
                                                  whole_skeletons_for_stitching,
                                                  stitch_print = False,
                                                  main_mesh = []
                                                )
final_skeleton_clean = sk.clean_skeleton(final_skeleton_pre_clean)

In [ ]:
final_skeleton_clean.shape

In [ ]:
sk.graph_skeleton_and_mesh(
                            main_mesh_total.vertices,
                          main_mesh_total.faces,
                          edge_coordinates=final_skeleton_clean
                          )

In [ ]:
sk = reload(sk)

In [ ]:
clean_time = time.time()
new_cleaned_skeleton = sk.clean_skeleton(final_skeleton_clean,
                        distance_func=sk.skeletal_distance,
                  min_distance_to_junction=5000,
                  return_skeleton=True,
                  print_flag=False)
print(f"Total time for skeleton clean {time.time() - clean_time}")

In [ ]:
sk.graph_skeleton_and_mesh(
                            main_mesh_total.vertices,
                          main_mesh_total.faces,
                          edge_coordinates=new_cleaned_skeleton,
                            axis_box_off = True
                          )


# Saving the Skeleton

In [ ]:
sk.save_skeleton_cgal(new_cleaned_skeleton,f"{current_name}_sk_cleaned.cgal")

# Saving as an HTML file

In [ ]:
import ipyvolume as ipv
ipv.pylab.save("Dusting_cleaned_5000.html")